In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
import math
from copy import deepcopy
import itertools
from collections import Counter

In [ ]:
# dodac produkcje zwracaja nowe pod grafy <check> 
# dodac do node'ow typu I id parenta, pierwszy node Typu I bedzie zawierac parenta -1 <check>
# layer ma byc lista <check>
# wizualizacja warstwy (brutem) <check>
# dodac check czy mozna wywolac <check>
#czyscic kernel i out przed pushem

In [ ]:
class Id_creator:
    def __init__(self):
        self.last_id = -1
    def get_id(self):
        self.last_id +=1
        return self.last_id
    def __call__(self):
        return self.get_id()
    
    
class CannotExecuteProduction(Exception):
        pass
    
    
class Graph_layers:
    def __init__(self):
        self._node_id_gen =  Id_creator()
        G = nx.Graph()
        G.add_nodes_from([(self._node_id_gen(), {'pos': (0,0),'type': 'e'})])
        self._layers=[[G]]
        
    def get_layer(self, i):
        return self._layers[i]
    
    def add_to_layer(self,i,G):
        return self._layers[i].append(G)
    
    def add_to_last_layer(self,G):
        return self._layers[-1].append(G)
        
    def add_new_layer(self, G_new):
        self._layers.append([G_new])            
        
    def get_last_layer_index(self):
        return len(self._layers) - 1
    
    def get_last_layer(self,i):
        return self._layers[i]
    
    def display_i_layer(self, i):
        G_layer = self._layers[i]
        G = self._layers[i][0] if  len(self._layers[i])==1 else reduce(nx.algorithms.operators.binary.compose,G_layer)
        
        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx(G, pos)
        
    def display_layer(self, layer):
        G_layer = layer
        G = layer[0] if  len(layer)==1 else reduce(nx.algorithms.operators.binary.compose,G_layer)
        
        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx(G, pos)
        
    def get_node_id_gen(self):
        return self._node_id_gen

In [ ]:
def get_graph_i_nodes(G):
    return [key for (key, value) in nx.get_node_attributes(G, 'type').items() if value == 'I']

def get_graph_i_nodes_by_ids(G, ids):
    return [key for (key, value) in nx.get_node_attributes(G, 'type').items() if value == 'I' and key in ids]

def get_parent_of_nodes(G, nodes):
    return list(set([value for (key, value) in nx.get_node_attributes(G, 'parent').items() if key in nodes]))

def get_distance(fst, snd):
    return math.sqrt((fst[0]-snd[0])**2 + (fst[1]-snd[1])**2)

In [ ]:
def p1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
    assert(G.nodes[base_node_id]['type'].lower() == 'e' )
    assert(len(G.nodes)==1)
    all_new_nodes = []
    all_new_edges = []
    base_pos = G.nodes[base_node_id]['pos']
    x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    i_node_x, i_node_y = base_pos[0], base_pos[1]
    i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
    all_new_nodes.append(i_node)
    
    half_side_len = side_len/2
    e_nodes = [
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'})
    ]
    for i in range(len(e_nodes)):
        all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
        all_new_edges.extend([(i_node[0],e_nodes[i][0])])
    
    all_new_nodes.extend(e_nodes)
    nG = nx.Graph()
    nG.add_nodes_from(all_new_nodes)
    nG.add_edges_from(all_new_edges)
    return nG, i_node


In [ ]:
def conc_duplicates(G):
    org_nodes = list(G.nodes(data=True))

    org_nodes_dict = {}
    for node_id,data in org_nodes:
        try:
            org_nodes_dict[data['pos']].append(node_id)
        except KeyError:
            org_nodes_dict[data['pos']]=[node_id]

    nodes_to_replace = {} #keys nodes to be deleted, values nodes to replace them
    for key,ids in org_nodes_dict.items():
        if len(ids) > 1:
            for id_ in ids[1:]:
                nodes_to_replace[id_] = ids[0]
                
    for old_node, new_node in nodes_to_replace.items():
        neighbors = list(G.neighbors(old_node))
        for n in neighbors:
            G.remove_edge(old_node,n)
            G.add_edge(new_node,n)
        G.remove_node(old_node)
    return G

In [ ]:
def avg_pos(pos1, pos2):
    return ((pos1[0] + pos2[0])/2, (pos1[1] + pos2[1])/2)

def rm_edge_if_exists(G,n1id,n2id):
    try:
        G.remove_edge(n1id,n2id)
    except nx.NetworkXError:
        pass
    


#     e0 - - - e1
#      | \    / |
#      |  I0    |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - n0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    n1 - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - n4 - - - e3

def p2(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    if len(es) != 4:
        raise CannotExecuteProduction
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    if not(G.has_edge(e0[0],e1[0]) and\
            G.has_edge(e1[0],e3[0]) and\
            G.has_edge(e3[0],e2[0]) and\
            G.has_edge(e0[0],e2[0]) and\
            G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
    #prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG
#     conc_duplicates(G)


In [ ]:
graph_layers = Graph_layers()
graph_layers.display_i_layer(0)

In [ ]:
base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G = graph_layers.get_layer(0)[0].copy()

#apply p1 production
G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
graph_layers.add_new_layer(G)

graph_layers.display_i_layer(1)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(1)[0]
nG = p2(G, 1, graph_layers.get_node_id_gen())
graph_layers.add_new_layer(nG)
graph_layers._layers[2][0].nodes(data=True)
graph_layers.display_i_layer(2)

In [ ]:
#apply p2 production 4 times
G = graph_layers.get_layer(2)[0]
graph_layers.add_new_layer( p2(G, 15, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 18, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 17, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 16, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#
#      I1  - e1    e1 - I3
#        \\  /        \\ /
#         e2          e2
#        / \\         / \\
#     I2  - e3     e3 - I4
#
#              |
#             \\/
#
#      I1  - e1 - I3
#        \\   |   /
#            e2
#        /   |   \\
#     I2  - e3 - I4
def p7(parent_layer, child_layer, base_node_ids, n_id_gen):
    #finding graph in child_layer
    first, second = None, None
    for graph in child_layer:
        if any(item in graph.nodes for item in base_node_ids):
            if second != None:
                raise CannotExecuteProduction
            if first is None:
                first = graph
            else:
                second = graph

    first_i_nodes = get_graph_i_nodes_by_ids(first, base_node_ids)
    second_i_nodes = get_graph_i_nodes_by_ids(second, base_node_ids)

    #checking each I node has a parent
    if not all('parent' in first.node[n] for n in first_i_nodes) or not all('parent' in second.node[n] for n in second_i_nodes):
        raise CannotExecuteProduction

    #finding graph in parent_layer
    parents = get_parent_of_nodes(first, first_i_nodes) + get_parent_of_nodes(second, second_i_nodes)
    if len(parents) != 2:
        raise CannotExecuteProduction
    parent_graphs = [graph for graph in parent_layer if all(parent in graph.nodes for parent in parents)]
    if len(parent_graphs) > 1:
        raise CannotExecuteProduction
    parent_graph = parent_graphs[0]
    if not any(fst for fst in parent_graph.neighbors(parents[0]) for snd in parent_graph.neighbors(parents[1]) if fst == snd):
        raise CannotExecuteProduction

    #finding nodes to reduce
    nodes_to_reduce_all = [(x_n, y_n) 
         for x in first_i_nodes 
         for y in second_i_nodes 
         for x_n in first.neighbors(x) 
         for y_n in second.neighbors(y)
         if 0.1 > get_distance(nx.get_node_attributes(first, 'pos')[x_n], nx.get_node_attributes(second, 'pos')[y_n])]
    nodes_to_reduce = list(set(nodes_to_reduce_all))
    
    if len(nodes_to_reduce_all) != 6 or len(nodes_to_reduce) != 3:
        raise CannotExecuteProduction
    
    possible_edges = [True for (x, _) in nodes_to_reduce for (y, _) in nodes_to_reduce if x != y and first.has_edge(x, y)]
    if len(possible_edges) != 4:
        raise CannotExecuteProduction
    possible_edges = [True for (_, x) in nodes_to_reduce for (_, y) in nodes_to_reduce if x != y and second.has_edge(x, y)]
    if len(possible_edges) != 4:
        raise CannotExecuteProduction
        
    
    #creating new correct graph
    reversed_nodes_to_reduce = [(value, key) for (key, value) in nodes_to_reduce]
    mapping = dict(nodes_to_reduce + reversed_nodes_to_reduce)

    new_old_graph = deepcopy(first)
    second_old_graph = nx.relabel_nodes(second, mapping, copy=True)

    nodes_to_add = [(node, values)
                             for (node, values) in second_old_graph.nodes().items() 
                             if node not in mapping.keys()]

    new_old_graph.add_nodes_from(nodes_to_add)
    new_old_graph.add_edges_from(second_old_graph.edges)

    i_nodes = get_graph_i_nodes(new_old_graph)

    return i_nodes, new_old_graph, [first, second]

In [ ]:
def apply_P7(parent_layer, child_layer, base_node_ids, n_id_gen):
    i_nodes, new_graph, old_graphs = p7(parent_layer, child_layer, base_node_ids, n_id_gen)
    for graph in old_graphs:
        child_layer.remove(graph)
    child_layer.append(new_graph)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [41, 43, 55, 57], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [30, 31, 55, 54], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
apply_P7(graph_layers.get_layer(2), graph_layers.get_layer(3), [29, 31, 67, 69], graph_layers.get_node_id_gen())
graph_layers.display_i_layer(3)

In [ ]:
def create_leaf_edges(parent, id_gen, offset, tiny_offset):
    center_x, center_y = 2, 2
    l2 = nx.Graph()
    l2_edges = []
    l2_nodes = []
    upper_i = (id_gen.get_id(), {'pos': (center_x + offset, center_y + 1),'type': 'I', 'parent': parent[0]})
    lower_i = (id_gen.get_id(), {'pos': (center_x + offset, center_y - 1),'type': 'I', 'parent': parent[0]})
    l2_nodes.extend([upper_i, lower_i])

    a_upper_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y + 1.5),'type': 'e'})
    a_mid_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y),'type': 'e'})
    a_lower_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y - 1.5),'type': 'e'})
    b_upper_e = (id_gen.get_id(), {'pos': (center_x + 2*offset, center_y + 1.5),'type': 'e'})
    b_mid_e = (id_gen.get_id(), {'pos': (center_x + 2*offset, center_y),'type': 'e'})
    b_lower_e = (id_gen.get_id(), {'pos': (center_x + 2*offset, center_y - 1.5),'type': 'e'})

    l2_nodes.extend([a_upper_e, a_mid_e, a_lower_e, b_upper_e, b_mid_e, b_lower_e])
    l2_edges.extend([(a_upper_e[0], a_mid_e[0]), 
                     (a_lower_e[0], a_mid_e[0]), 
                     (a_lower_e[0], lower_i[0]),
                     (lower_i[0], a_mid_e[0]),
                     (upper_i[0], a_mid_e[0]),
                     (upper_i[0], a_upper_e[0]),
                     (b_upper_e[0], b_mid_e[0]), 
                     (b_lower_e[0], b_mid_e[0]), 
                     (b_lower_e[0], lower_i[0]),
                     (lower_i[0], b_mid_e[0]),
                     (upper_i[0], b_mid_e[0]),
                     (upper_i[0], b_upper_e[0]),
                     (b_upper_e[0], a_upper_e[0]),
                     (b_mid_e[0], a_mid_e[0]),
                     (b_lower_e[0], a_lower_e[0])])

    l2.add_nodes_from(l2_nodes)
    l2.add_edges_from(l2_edges)
    return l2, [lower_i[0], upper_i[0]]

In [ ]:

def create_valid_input_net():
    graph_layers = Graph_layers()
    side_len=2
    parent_node_x, parent_node_y = 1, 1
    id_gen = graph_layers.get_node_id_gen()
    parent_node = (id_gen.get_id(), {'pos': (parent_node_x, parent_node_y), 'type': 'E','parent': -1})

    half_side_len = side_len/2
    i_nodes = [
        (id_gen.get_id(), {'pos': (parent_node_x + half_side_len, parent_node_y - half_side_len),'type': 'I', 'parent': parent_node[0]}),
        (id_gen.get_id(), {'pos': (parent_node_x - half_side_len, parent_node_y - half_side_len),'type': 'I', 'parent': parent_node[0]})
    ]
    l1_edges = []
    for i_node in i_nodes:
         l1_edges.extend([(i_node[0],parent_node[0])])

    l1 = nx.Graph()
    l1.add_nodes_from([parent_node])
    l1.add_nodes_from(i_nodes)
    l1.add_edges_from(l1_edges)
    graph_layers.add_new_layer(l1)

    center_x, center_y, offset, tiny_offset = 2, 2, -1, -0.05
    left_graph, output_ids = create_leaf_edges(i_nodes[0], id_gen, offset, tiny_offset)
    graph_layers.add_new_layer(left_graph)
    offset = -offset
    tiny_offset = -tiny_offset
    right_graph, right_ids = create_leaf_edges(i_nodes[1], id_gen, offset, tiny_offset)
    graph_layers.add_to_layer(2, right_graph)
    output_ids.extend(right_ids)

    return graph_layers, output_ids



In [ ]:
# obraz pogladowy jasne jest ze istnieja 4 permutacje izomorficzne jesli chodzi o labelkowanie wierzcholkow
#     e0 - - - e1
#      | \    / |
#     e4   I0   |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0' - - - n0 - - - e1'
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    e4' - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2' - - - n4 - - - e3'
def p3(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    nghs_counter = Counter(reduce(lambda a,b : a+b ,[list(G.neighbors(e)) for e in es],es + [base_node_id]))
    es1 = set([x for x, count in nghs_counter.items() if count >= 2]) - {base_node_id}
    if len(es) != 4 and len(es1) != 5:
        raise CannotExecuteProduction
    e0, e1, e2, e3, e4 = None, None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    
    
    # get Square corners
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    ids = [base_node_id, e0[0], e1[0], e2[0], e3[0]]

    edge0 = G.has_edge(e0[0],e1[0])
    edge1 = G.has_edge(e0[0],e2[0])
    edge2 = G.has_edge(e2[0],e3[0])
    edge3 = G.has_edge(e1[0],e3[0])
    
    # should be 3 edges between 4 cornesrs (1 is created with extra node)
    if(int(edge0) + int(edge1) + int(edge2) + int(edge3) != 3):
         raise CannotExecuteProduction
           
    def getNodeBetween(e1Id, e2Id):
        allNodes = list(G.neighbors(e1Id)) + list(G.neighbors(e2Id))
        possibleNodes = [n for n in allNodes if n in list(G.neighbors(e1Id)) and n in list(G.neighbors(e2Id))]
        return list(set(possibleNodes))
    
    if(not edge0):
        possE4 = getNodeBetween(e0[0],e1[0])
    if(not edge1):
        possE4 = getNodeBetween(e0[0],e2[0])
    if(not edge2):
        possE4 = getNodeBetween(e2[0],e3[0])
    if(not edge3):
        possE4 = getNodeBetween(e1[0],e3[0])
 
    # get all 'e' nodes between two nodes that does not share edge
    maybeE4 = [ G.nodes[e] for e in possE4 if G.nodes[e]['type'].lower() == 'e']
    if(len(maybeE4) != 1):
        raise CannotExecuteProduction
    
    # this is the extra node
    e4 = maybeE4[0]

    #check if node has proper cordinates
    if(not edge0):
        if(e4['pos'] != avg_pos(e0[1]['pos'], e1[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge1):
        if(e4['pos'] != avg_pos(e0[1]['pos'], e2[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge2):
        if(e4['pos'] != avg_pos(e2[1]['pos'], e3[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge3):
        if(e4['pos'] != avg_pos(e1[1]['pos'], e3[1]['pos'])):
            raise CannotExecuteProduction
                                                                                     
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG


In [ ]:
# obraz pogladowy jasne jest ze istnieja 4 permutacje izomorficzne jesli chodzi o labelkowanie wierzcholkow
#     e0 - e5 - e1
#      | \    / |
#     e4   I0   |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0' - - - n0 - - - e1'
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    e4' - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2' - - - n4 - - - e3'
def p4(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    nghs_counter = Counter(reduce(lambda a,b : a+b ,[list(G.neighbors(e)) for e in es],es + [base_node_id]))
    es1 = set([x for x, count in nghs_counter.items() if count >= 2]) - {base_node_id}
    if len(es) != 4 and len(es1) != 6:
        raise CannotExecuteProduction
    e0, e1, e2, e3, e4, e5 = None, None, None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    
    
    # get Square corners
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    ids = [base_node_id, e0[0], e1[0], e2[0], e3[0]]

    edge0 = G.has_edge(e0[0],e1[0])
    edge1 = G.has_edge(e0[0],e2[0])
    edge2 = G.has_edge(e2[0],e3[0])
    edge3 = G.has_edge(e1[0],e3[0])
    
    # should be 3 edges between 4 cornesrs (1 is created with extra node)
    if(int(edge0) + int(edge1) + int(edge2) + int(edge3) != 2):
         raise CannotExecuteProduction
    # 'edges' with addtional node may lay near each other (cannot be on the other side of square)
    if(int(edge0) + int(edge2) == 2):
        raise CannotExecuteProduction
    if(int(edge1) + int(edge3) == 2):
        raise CannotExecuteProduction
           
    def getNodeBetween(e1Id, e2Id):
        allNodes = list(G.neighbors(e1Id)) + list(G.neighbors(e2Id))
        possibleNodes = [n for n in allNodes if n in list(G.neighbors(e1Id)) and n in list(G.neighbors(e2Id))]
        return list(set(possibleNodes))
    
    possE4andE5 = list([])
    if(not edge0):
        possE4andE5 = possE4andE5 + getNodeBetween(e0[0],e1[0])
    if(not edge1):
        possE4andE5 = possE4andE5 + getNodeBetween(e0[0],e2[0])
    if(not edge2):
        possE4andE5 = possE4andE5 + getNodeBetween(e2[0],e3[0])
    if(not edge3):
        possE4andE5 = possE4andE5 + getNodeBetween(e1[0],e3[0])
 
    # get all 'e' nodes between two nodes that does not share edge
    maybeE4AndE5 = [ G.nodes[e] for e in possE4andE5 if G.nodes[e]['type'].lower() == 'e']
    if(len(maybeE4AndE5) != 2):
        raise CannotExecuteProduction
    
    # these are the extra nodes
    e4 = maybeE4AndE5[0]
    e5 = maybeE4AndE5[0]
    
    def checkIfHasOneOfProperPositionsOrRaise(e):
        proper = False
        if(not edge0):
            if(e['pos'] == avg_pos(e0[1]['pos'], e1[1]['pos'])):
                proper = True
        if(not edge1):
            if(e['pos'] != avg_pos(e0[1]['pos'], e2[1]['pos'])):
                proper = True
        if(not edge2):
            if(e['pos'] != avg_pos(e2[1]['pos'], e3[1]['pos'])):
                proper = True
        if(not edge3):
            if(e['pos'] != avg_pos(e1[1]['pos'], e3[1]['pos'])):
                proper = True
        if(not proper):
            raise CannotExecuteProduction
        
    
    #check if node has proper cordinates
    checkIfHasOneOfProperPositionsOrRaise(e4)
    checkIfHasOneOfProperPositionsOrRaise(e5)
    
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG

In [ ]:
def zad_2_tests():
    
    def print_res(graph_layers,prod_f, prod,start_layer):
        fig1 = plt.figure(1)
        graph_layers.display_i_layer(start_layer)
        fig1.suptitle(f'base {prod} graph')
        plt.show()
        print(f'Applying {prod} production')
        graph_layers.add_new_layer(prod_f())
        fig2 = plt.figure(2)
        fig2.suptitle(f'production {prod} result')
        graph_layers.display_i_layer(start_layer+1)
        plt.show()
        print('Success')
        print()
    
    def prepare_graph(f=None):
        graph_layers = Graph_layers()
        base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
        first_layer_G = graph_layers.get_layer(0)[0].copy()
        if f:
            G, i_node = f(first_layer_G, base_node[0], graph_layers.get_node_id_gen())
            graph_layers.add_new_layer(G)
        else:
            G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
            graph_layers.add_new_layer(G)
            G = graph_layers.get_layer(1)[0]
            G = p2(G, 1, graph_layers.get_node_id_gen())
            graph_layers.add_new_layer(G)
            graph_layers._layers[2][0].nodes(data=True)
        return graph_layers,G,i_node
        
        
    def pX1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
        assert(G.nodes[base_node_id]['type'].lower() == 'e' )
        assert(len(G.nodes)==1)
        all_new_nodes = []
        all_new_edges = []
        base_pos = G.nodes[base_node_id]['pos']
        x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        i_node_x, i_node_y = base_pos[0], base_pos[1]
        i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
        all_new_nodes.append(i_node)

        half_side_len = side_len/2
        e_nodes = [
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y),'type': 'e'})
        ]
        for i in range(len(e_nodes)):
            all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
            if i < 4:
                all_new_edges.extend([(i_node[0],e_nodes[i][0])])

        all_new_nodes.extend(e_nodes)
        nG = nx.Graph()
        nG.add_nodes_from(all_new_nodes)
        nG.add_edges_from(all_new_edges)
        return nG, i_node
    
    def pX2(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
        assert(G.nodes[base_node_id]['type'].lower() == 'e' )
        assert(len(G.nodes)==1)
        all_new_nodes = []
        all_new_edges = []
        base_pos = G.nodes[base_node_id]['pos']
        x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        i_node_x, i_node_y = base_pos[0], base_pos[1]
        i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
        all_new_nodes.append(i_node)

        half_side_len = side_len/2
        e_nodes = [
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (7, {'pos': (i_node_x, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (6, {'pos': (i_node_x - half_side_len, i_node_y),'type': 'e'})
        ]
        for i in range(len(e_nodes)):
            all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
            if i != 1 and i != 5:
                all_new_edges.extend([(i_node[0],e_nodes[i][0])])

        all_new_nodes.extend(e_nodes)
        nG = nx.Graph()
        nG.add_nodes_from(all_new_nodes)
        nG.add_edges_from(all_new_edges)
        return nG, i_node
    
    #TC1
    print('p3 simple graph')
    graph_layers, G, i_node = prepare_graph(pX1)
    
    print("Trying to use p4 on p3 graph")
    try:
        p4(G, i_node[0], graph_layers.get_node_id_gen()) 
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")

    print_res(graph_layers,lambda :p3(G, i_node[0], graph_layers.get_node_id_gen()),'p3',1)
    
    #TC2
    print('p4 simple graph')
    graph_layers, G, i_node = prepare_graph(pX2)
    
    print("Trying to use p3 on p4 graph")
    try:
        p3(G, i_node[0], graph_layers.get_node_id_gen()) 
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
        
    print_res(graph_layers,lambda :p4(G, i_node[0], graph_layers.get_node_id_gen()), 'p4',1)
    
    #TC3
    print('p3 not so simple graph')
    graph_layers, G, i_node = prepare_graph()
    rm_edge_if_exists(G, 6, 11)
    rm_edge_if_exists(G, 6, 10)
    n6 = list(G.nodes(data=True))[0]
    n10 = list(G.nodes(data=True))[4]
    n11 = list(G.nodes(data=True))[5]
    n19 = (19, {'pos' : avg_pos(n6[1]['pos'],n11[1]['pos']),'type':'e'})
    n20 = (20, {'pos' : avg_pos(n6[1]['pos'],n10[1]['pos']),'type':'e'})
    G.add_nodes_from([n19, n20])
    G.add_edges_from([(n6[0], n19[0]), (n19[0], n11[0]), (n20[0], n6[0]), (n10[0], n20[0])])
    
    print("Trying to use p3 on p4 graph")
    try:
        p3(G, 15, graph_layers.get_node_id_gen())
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
    
    print_res(graph_layers,lambda :p4(G, 15, graph_layers.get_node_id_gen()),'p4',2)
    
    #TC4
    print('p4 not so simple graph')
    graph_layers, G, i_node = prepare_graph()
    rm_edge_if_exists(G, 6, 11)
    n6 = list(G.nodes(data=True))[0]
    n11 = list(G.nodes(data=True))[5]
    n19 = (19, {'pos' : avg_pos(n6[1]['pos'],n11[1]['pos']),'type':'e'})
    G.add_nodes_from([n19])
    G.add_edges_from([(n6[0], n19[0]), (n19[0], n11[0])])
    
    print("Trying to use p4 on p3 graph")
    try:
        p4(G, 15, graph_layers.get_node_id_gen())
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
    
    print_res(graph_layers, lambda :p3(G, 15, graph_layers.get_node_id_gen()),'p3',2)
zad_2_tests()

In [ ]:
import unittest
import random

class TestNotebook(unittest.TestCase):

    def create_valid_output_layer(self):
        graph_layers = Graph_layers()
        id_gen = graph_layers.get_node_id_gen()
        l2 = nx.Graph()
        l2_edges = []
        l2_nodes = []
        center_x, center_y, offset, = 2, 2, -1
        upper_i_left = (id_gen.get_id(), {'pos': (center_x + offset, center_y + 1),'type': 'I', 'parent': 2})
        lower_i_left = (id_gen.get_id(), {'pos': (center_x + offset, center_y - 1),'type': 'I', 'parent': 2})
        offset = -offset
        upper_i_right = (id_gen.get_id(), {'pos': (center_x + offset, center_y + 1),'type': 'I', 'parent': 3})
        lower_i_right = (id_gen.get_id(), {'pos': (center_x + offset, center_y - 1),'type': 'I', 'parent': 3})

        l2_nodes.extend([upper_i_right, upper_i_left, lower_i_left, lower_i_right])

        upper_e = (id_gen.get_id(), {'pos': (center_x, center_y + 1.5),'type': 'e'})
        mid_e = (id_gen.get_id(), {'pos': (center_x, center_y),'type': 'e'})
        lower_e = (id_gen.get_id(), {'pos': (center_x, center_y - 1.5),'type': 'e'})

        l2_nodes.extend([upper_e, mid_e, lower_e])
        l2_edges.extend([(upper_e[0], mid_e[0]), (lower_e[0], mid_e[0]),
                         (lower_e[0], lower_i_left[0]),
                         (lower_e[0], lower_i_right[0]),
                         (lower_i_left[0], mid_e[0]),
                         (lower_i_right[0], mid_e[0]),
                         (upper_i_left[0], mid_e[0]),
                         (upper_i_right[0], mid_e[0]),
                         (upper_i_right[0], upper_e[0]),
                         (upper_i_left[0], upper_e[0])])

        l2.add_nodes_from(l2_nodes)
        l2.add_edges_from(l2_edges)
        graph_layers.add_new_layer(l2)
        # graph_layers.display_i_layer(1)
        return graph_layers.get_layer(1)[0]


    def create_leaf_edges(self, parent, id_gen, offset, tiny_offset):
        center_x, center_y = 2, 2
        l2 = nx.Graph()
        l2_edges = []
        l2_nodes = []
        upper_i = (id_gen.get_id(), {'pos': (center_x + offset, center_y + 1),'type': 'I', 'parent': parent[0]})
        lower_i = (id_gen.get_id(), {'pos': (center_x + offset, center_y - 1),'type': 'I', 'parent': parent[0]})
        l2_nodes.extend([upper_i, lower_i])

        upper_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y + 1.5),'type': 'e'})
        mid_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y),'type': 'e'})
        lower_e = (id_gen.get_id(), {'pos': (center_x + tiny_offset, center_y - 1.5),'type': 'e'})

        l2_nodes.extend([upper_e, mid_e, lower_e])
        l2_edges.extend([(upper_e[0], mid_e[0]), (lower_e[0], mid_e[0]), (lower_e[0], lower_i[0]),
                         (lower_i[0], mid_e[0]),
                         (upper_i[0], mid_e[0]),
                         (upper_i[0], upper_e[0])])

        l2.add_nodes_from(l2_nodes)
        l2.add_edges_from(l2_edges)
        return l2, [lower_i[0], upper_i[0]]

    def create_valid_input_net(self):
        graph_layers = Graph_layers()
        side_len=2
        parent_node_x, parent_node_y = 1, 1
        id_gen = graph_layers.get_node_id_gen()
        parent_node = (id_gen.get_id(), {'pos': (parent_node_x, parent_node_y), 'type': 'E','parent': -1})

        half_side_len = side_len/2
        i_nodes = [
            (id_gen.get_id(), {'pos': (parent_node_x + half_side_len, parent_node_y - half_side_len),'type': 'I', 'parent': parent_node[0]}),
            (id_gen.get_id(), {'pos': (parent_node_x - half_side_len, parent_node_y - half_side_len),'type': 'I', 'parent': parent_node[0]})
        ]
        l1_edges = []
        for i_node in i_nodes:
             l1_edges.extend([(i_node[0],parent_node[0])])

        l1 = nx.Graph()
        l1.add_nodes_from([parent_node])
        l1.add_nodes_from(i_nodes)
        l1.add_edges_from(l1_edges)
        graph_layers.add_new_layer(l1)

        center_x, center_y, offset, tiny_offset = 2, 2, -1, -0.05
        left_graph, output_ids = self.create_leaf_edges(i_nodes[0], id_gen, offset, tiny_offset)
        graph_layers.add_new_layer(left_graph)
        offset = -offset
        tiny_offset = -tiny_offset
        right_graph, right_ids = self.create_leaf_edges(i_nodes[1], id_gen, offset, tiny_offset)
        graph_layers.add_to_layer(2, right_graph)
        output_ids.extend(right_ids)

#         graph_layers.display_i_layer(2)

        return graph_layers, output_ids

    def test_p7_correct(self):
        initial, intermediate_i_node_ids = self.create_valid_input_net()
        output = p7(initial.get_layer(1), initial.get_layer(2), intermediate_i_node_ids, initial.get_node_id_gen())
        assert nx.is_isomorphic(self.create_valid_output_layer(), output[1], node_match=lambda n1,n2: n1['type'] == n2['type'] and n1.get('parent', -2) == n2.get('parent', -2))


    def test_p7_remove_parent_attribute(self):
        layers, intermediate_i_node_ids = self.create_valid_input_net()
        left_side = layers.get_layer(2)[0]
        right_side = layers.get_layer(2)[1]
        nodes = list(filter(lambda n: n in intermediate_i_node_ids, left_side.nodes))

        selected_node = left_side.node[nodes[0]]
        del selected_node['parent']

        with self.assertRaises(CannotExecuteProduction):
            p7(layers.get_layer(1), [left_side, right_side], intermediate_i_node_ids, layers.get_node_id_gen())


    def test_p7_remove_random_edge(self):
        layers, intermediate_i_node_ids = self.create_valid_input_net()
        left_side = layers.get_layer(2)[0]
        right_side = layers.get_layer(2)[1]
        edges = list(left_side.edges)

        # random edge choice
        chosen_edge = random.choice(edges)
        left_side.remove_edge(chosen_edge[0], chosen_edge[1])

        with self.assertRaises(CannotExecuteProduction):
            p7(layers.get_layer(1), [left_side, right_side], intermediate_i_node_ids, layers.get_node_id_gen())


    def test_p7_remove_random_node(self):
        layers, intermediate_i_node_ids = self.create_valid_input_net()
        left_side = layers.get_layer(2)[0]
        right_side = layers.get_layer(2)[1]
        nodes = list(left_side.nodes)

        # random node choice
        left_side.remove_node(random.choice(nodes))

        with self.assertRaises(CannotExecuteProduction):
            p7(layers.get_layer(1), [left_side, right_side], intermediate_i_node_ids, layers.get_node_id_gen())


unittest.main(argv=[''], verbosity=2, exit=False)



In [ ]:
#     e0 - m0 - e1
#      | \    / |
#     m1   I0   m2 
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - m0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    m1 - - - n0 - - - m2
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - m3 - - - e3
def p5(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    
    # assign e
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
                
    # check if all e are present
    if not e0 or not e2 or not e2 or not e3:
        raise CannotExecuteProduction
    
    # edges between e - e
    edge0 = G.has_edge(e0[0],e1[0])
    edge1 = G.has_edge(e0[0],e2[0])
    edge2 = G.has_edge(e1[0],e3[0])
    edge3 = G.has_edge(e2[0],e3[0])
    
    # check I - e edges
    if not(G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
        
    # assign m0, m1, m2, m3
    # m's are only assigned if they are e0 or e3 neighbour list and have proper x, y
    m0, m1, m2, m3 = None, None, None, None
    e0_neigh = list(G.neighbors(e0[0]))
    e3_neigh = list(G.neighbors(e3[0]))
    e0X, e0Y = e0[1]['pos']
    e1X, e1Y = e1[1]['pos']
    e2X, e2Y = e2[1]['pos']
    e3X, e3Y = e3[1]['pos']
    for node_x in e0_neigh:
        x,y = G.nodes[node_x]['pos']
        if x==(e0X+e1X)/2 and y==(e0Y+e1Y)/2:
            m0 = (node_x,G.nodes[node_x])
        if x==(e0X+e2X)/2 and y==(e0Y+e2Y)/2:
            m1 = (node_x,G.nodes[node_x])
    for node_x in e3_neigh:
        x,y = G.nodes[node_x]['pos']
        if x==(e1X+e3X)/2 and y==(e1Y+e3Y)/2:
            m2 = (node_x,G.nodes[node_x])
        if x==(e2X+e3X)/2 and y==(e2Y+e3Y)/2:
            m3 = (node_x,G.nodes[node_x])
    
    # check if all three m present
    m_list = [m0, m1, m2, m3]
    none_m_num = sum(x is None for x in m_list)
    if none_m_num != 1:
        raise CannotExecuteProduction
        
    # check if m not present the e-e edge exists
    if m0 is None and not edge0:
        raise CannotExecuteProduction
    if m1 is None and not edge1:
        raise CannotExecuteProduction
    if m2 is None and not edge2:
        raise CannotExecuteProduction
    if m3 is None and not edge3:
        raise CannotExecuteProduction
        
    # prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    # generate m if not exists
    if m0 is None:
        m0 = (n_id_gen(),{'pos': avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    else:
        m0 = (n_id_gen(),{'pos': m0[1]['pos'],'type': m0[1]['type']})
    
    if m1 is None:
        m1 = (n_id_gen(),{'pos': avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    else:
        m1 = (n_id_gen(),{'pos': m1[1]['pos'],'type': m1[1]['type']})
    
    if m2 is None:
        m2 = (n_id_gen(),{'pos': avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    else:
        m2 = (n_id_gen(),{'pos': m2[1]['pos'],'type': m2[1]['type']})
    
    if m3 is None:
        m3 = (n_id_gen(),{'pos': avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    else:
        m3 = (n_id_gen(),{'pos': m3[1]['pos'],'type': m3[1]['type']})
        
    n0 = (n_id_gen(),{'pos': avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos': avg_pos(e0[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos': avg_pos(e1[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos': avg_pos(e2[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos': avg_pos(e3[1]['pos'],n0[1]['pos']),'type':'I','parent':base_node_id})
    
    # add all new edges
    new_edges = [
        (e0[0],m0[0]),(m0[0],e1[0]),
        (e0[0],I1[0]),(m0[0],I1[0]), (m0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],m1[0]),(m0[0],n0[0]),(e1[0],m2[0]),
        (m1[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(m2[0],I2[0]),
        (m1[0],n0[0]),(n0[0],m2[0]),
        (m1[0],I3[0]),(n0[0],I3[0]), (n0[0],I4[0]),(m2[0],I4[0]),
        (m1[0],e2[0]),(n0[0],m3[0]),(m2[0],e3[0]),
        (e2[0],I3[0]),(m3[0],I3[0]), (m3[0],I4[0]),(e3[0],I4[0]),
        (e2[0],m3[0]),(m3[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,m0,m1,m2,n0,m3,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG

In [ ]:
graph_layers_p5 = Graph_layers()
base_node_p5 = list(graph_layers_p5.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G_p5 = graph_layers_p5.get_layer(0)[0].copy()

#apply p1 production
G_p5, i_node = p1(first_layer_G_p5, base_node_p5[0], graph_layers_p5.get_node_id_gen()) 

#temper the graph
rm_edge_if_exists(G_p5, 2, 3)
rm_edge_if_exists(G_p5, 2, 5)
rm_edge_if_exists(G_p5, 3, 4)
e0 = (2, G_p5.nodes[2])
e1 = (3, G_p5.nodes[3])
e2 = (5, G_p5.nodes[5])
e3 = (4, G_p5.nodes[4])
m0 = (111,{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type': 'e'})
m1 = (222,{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type': 'e'})
m2 = (333,{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type': 'e'})
new_edges = [
     (e0[0], m1[0]), (m1[0], e2[0]),
     (e0[0], m0[0]), (m0[0], e1[0]),
     (e1[0], m2[0]), (m2[0], e3[0]),
    ]
G_p5.add_nodes_from([m0,m1,m2])
G_p5.add_edges_from(new_edges)
graph_layers_p5.add_new_layer(G_p5)
graph_layers_p5.display_i_layer(1)

In [ ]:
#apply p5 production
G_p5 = graph_layers_p5.get_layer(1)[0]
nG_p5 = p5(G_p5, 1, graph_layers_p5.get_node_id_gen())
graph_layers_p5.add_new_layer(nG_p5)
graph_layers_p5._layers[2][0].nodes(data=True)
graph_layers_p5.display_i_layer(2)

In [ ]:
# p5 tests
import unittest
from collections import defaultdict

def run_test_p5():
    idc = Id_creator()
    
    # do głupiego debugowania
    def draw(g):
        pos = nx.get_node_attributes(g, 'pos')
        nx.draw_networkx(g, pos)
    
    # funkcje pomocniczne, klasa z testami poniżej
    # returns Graph, I id, list of e ids
    def make_base_graph():
        ids = [idc() for x in range(5)]
        nodes = [[i, {'type': 'e'}] for i in ids]
        i = nodes[0]
        es = nodes[1:]
        i[1]['type'] = 'i'
        
        edges = []
        pairs = [[0,1],[0,2],[1,3],[2,3]]

        for e in es:
            edges.append((i[0], e[0]))

        for a,b in pairs:
            edges.append((es[a][0], es[b][0]))
        
        i[1]['pos'] = (0,0)
        poses = [(-1, 1), (1, 1), (-1, -1), (1, -1)]
        for idx, pos in enumerate(poses):
            es[idx][1]['pos'] = pos
        
        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)

        return g, ids[0], ids[1:]
    
    def make_basic_graph(edges=[[0,1], [0,2], [1,3]]):
        g, i, es = make_base_graph()
        for ai, bi in edges:
            a, b = es[ai], es[bi]
            add_m_between(g, a, b)
        return (g, i, es)
    
    def add_m_between(g, ia, ib):
        m = idc()
        a = g.nodes[ia]['pos']
        b = g.nodes[ib]['pos']
        g.add_nodes_from([(m, {'type': 'e', 'pos': avg_pos(a,b)})])
        g.add_edges_from([[ia,m], [m,ib]])
        g.remove_edge(ia, ib)
        return m


    class TestProduction5(unittest.TestCase):
        def is_result_ok(self, g, base_node_id):
            def where_nodes(attr, t, nodes=None):
                ns = g.nodes if nodes == None else nodes
                attrs = nx.get_node_attributes(g, attr)
                return [n for n in ns if attrs[n] == t]

            def find_nodes_between(nodes):
                # for a given set of nodes [e0, e1, e2, ...]
                # find nodes (of type 'e') that are neighbors of exactly two of these nodes
                m_scores = defaultdict(int)
                for e in nodes:
                    for neighbor in g.neighbors(e):
                        if g.nodes[neighbor]['type'] == 'e':
                            m_scores[neighbor] += 1
                ms = [k for k, v in m_scores.items() if v == 2]
                return ms

            def is_node_between(m, a, b):
                expected_pos = avg_pos(g.nodes[a]['pos'], g.nodes[b]['pos'])
                return expected_pos == g.nodes[m]['pos']
    
            Is = where_nodes('parent', base_node_id, nodes=where_nodes('type', 'I'))

            # dla każdej ćwiartki
            for I in Is:
                es = list(g.neighbors(I))
                if len(es) != 4:
                    return False
                posc = lambda c: sum(g.nodes[n]['pos'][c] for n in es)/4
                # czy I znajduje się w średniej współrzędnych wszystkich sąsiadów
                self.assertEqual(g.nodes[I]['pos'], (posc(0), posc(1)))

            # czy pary I mają jednego/dwóch wspólnych sąsiadów
            for a, b in [[0,1], [0,2], [2,3], [1,3]]:
                I1, I2 = Is[a], Is[b]
                self.assertEqual(2, len(find_nodes_between([I1, I2])))
            for a, b in [[0,3], [1,2]]:
                I1, I2 = Is[a], Is[b]
                between = find_nodes_between([I1, I2])
                self.assertEqual(1, len(between))
                self.assertTrue(is_node_between(between[0], I1, I2))
            
            # czy I razem mają 9 e
            eses = [list(g.neighbors(I)) for I in Is]
            # flatten
            eses = [item for items in eses for item in items]
            eses = list(set(eses))
            self.assertEqual(len(eses), 9)
        

        def test_super_basic(self):
            # najprostszy test case
            g, i, es = make_basic_graph()
            p5_result = p5(g, i, idc)
            self.is_result_ok(p5_result, i)
            
            # odnajdź node po pozycji + asercja że istnieje dokładnie jeden
            def find_node(graph, pos):
                attrs = nx.get_node_attributes(graph, 'pos')
                results = [k for k, v in attrs.items() if v == pos]
                self.assertEqual(len(results), 1)
                return results[0]

            # sprawdzenie kilku nodów (górnych) i porównanie do lewej strony
            poses = nx.get_node_attributes(p5_result, 'pos')
            cord = lambda n: list(map(lambda pos: pos[n], poses.values()))
            topleft = find_node(p5_result, (min(cord(0)), max(cord(1))))
            topright = find_node(p5_result, (max(cord(0)), max(cord(1))))
            self.assertFalse(topleft in p5_result.neighbors(topright))
            
            # górny środkowy:
            topmid = find_node(p5_result, avg_pos(p5_result.nodes[topleft]['pos'], p5_result.nodes[topright]['pos']))
            self.assertTrue(topmid in p5_result.neighbors(topleft))
            self.assertTrue(topmid in p5_result.neighbors(topright))
            
            # czy istnieją odpowidniki po lewej stronie produkcji:
            
            org_topleft = find_node(g, p5_result.nodes[topleft]['pos'])
            org_topright = find_node(g, p5_result.nodes[topright]['pos'])
            org_topright = find_node(g, p5_result.nodes[topmid]['pos'])

        def test_subgraph_passes(self):
            # czy aplikuje dla podgrafu
            # dodaję ekstra wierzchołek i krawędź w kilku miejscach
            def add_to(g, n):
                some_e_id = es[ei]
                new_id = idc()
                g.add_nodes_from([(new_id, {'type': 'e', 'pos': (0, 0.5)})])
                g.add_edge(new_id, some_e_id)

            for ei in range(4):
                g, i, es = make_base_graph()
    
                for ai, bi in [[0,1], [0,2], [1,3]]:
                    a, b = es[ai], es[bi]
                    add_m_between(g, a, b)
                add_to(g, es[ei])
                self.is_result_ok(p5(g, i, idc), i)

            for m in range(3):
                g, i, es = make_base_graph()
                ms = []
                for ai, bi in [[0,1], [0,2], [1,3]]:
                    a, b = es[ai], es[bi]
                    ms.append(add_m_between(g, a, b))
                add_to(g, ms[m])
                self.is_result_ok(p5(g, i, idc), i)
            # kilka ekstra wierzchołków
            g, i, es = make_base_graph()
            for e in es:
                add_to(g, e)
            self.is_result_ok(p5(g, i, idc), i)


        def test_sides(self):
            g, i, es = make_basic_graph(edges=[[0,1], [0,2], [2,3]])
            self.is_result_ok(p5(g, i, idc), i)
            g, i, es = make_basic_graph(edges=[[0,1], [1,3], [2,3]])
            self.is_result_ok(p5(g, i, idc), i)

        def test_upsidedown(self):
            g, i, es = make_basic_graph(edges=[[0,2], [2,3], [1,3]])
            self.is_result_ok(p5(g, i, idc), i)

        # testy sprawdzające czy produkcja się nie wykona kiedy nie powinna:
        
        def test_complete(self):
            # przypadek dla grafu do lewej strony producji 6
            g, i, es = make_basic_graph(edges=[[0,1], [0,2], [2,3], [1,3]])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_mislabel(self):
            # na obecnosc etykiety type: e
            g, i, es = make_base_graph()
            for ei in range(4):
                e = es[ei]
                g.nodes[e]['type'] = 'qweqweqwe'
                with self.assertRaises(CannotExecuteProduction):
                    p5(g, i, idc)

        def test_missing_e(self):
            # po usunieciu node
            g, i, es = make_base_graph()
            for ei in range(4):
                g.remove_node(es[ei])
                with self.assertRaises(CannotExecuteProduction):
                    p5(g, i, idc)


        def test_missing_m(self):
            # przypadek dobry dla produkcji 4
            g, i, es = make_basic_graph(edges=[[0,2], [2,3]])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)

        def test_missing_edges(self):
            # brak krawędzi między I a e
            g, i, es = make_basic_graph()
            g.remove_edge(i, es[0])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
    
            # brak krawędzi między e a e
            g, i, es = make_basic_graph()
            g.remove_edge(es[2], es[3])
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_wrong_coords_e(self):
            g, i, es = make_basic_graph()
            g.nodes[es[0]]['pos'] = (10.0, 20.0)
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)
                
        def test_wrong_coords_i(self):
            g, i, es = make_basic_graph()
            g.nodes[i]['pos'] = (10.0, 20.0)
            with self.assertRaises(CannotExecuteProduction):
                p5(g, i, idc)

    suite = unittest.defaultTestLoader.loadTestsFromTestCase(TestProduction5)
    unittest.TextTestRunner().run(suite)
run_test_p5()